In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install seaborn

**Load the libraries and data**

In [ ]:
!pip install --upgrade pip

In [ ]:
# Load the required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import math
import re 
import os
import tensorflow as tf
from kaggle_datasets import KaggleDatasets

In [ ]:
print('tf ver:',tf.__version__)

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Load the required dataset

sample_submission = pd.read_csv("/kaggle/input/tpu-getting-started/sample_submission.csv")
sample_submission.head()

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) # what do gcs paths look like?

In [ ]:
IMAGE_SIZE = [512, 512]
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') 

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

# Data Pipelines

In [ ]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    
    # Random saturation
    image = tf.image.random_saturation(image, lower=0.8, upper=2)
    
    
    # Random brightness
#     image = tf.image.random_brightness(image, 0.2)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('{} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

In [ ]:
print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

In [ ]:
print("Test data shapes:")
for image, idnum in ds_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

# Data Exploration

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()


def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
ds_iter = iter(ds_train.unbatch().batch(15))

In [ ]:
one_batch = next(ds_iter)
display_batch_of_images(one_batch)

# Machine Learning
## Transfered Learning
### Model 1 - InceptionV3 model
Use pre-trained InceptionV3 Model with weight from ImageNet

In [ ]:
# with strategy.scope():
#     pretrained_model = tf.keras.applications.InceptionV3(
#         weights='imagenet',
#         include_top=False ,
#         input_shape=[*IMAGE_SIZE, 3]
#     )
# #     pretrained_model = tf.keras.applications.InceptionResNetV2(
# #             weights='imagenet',
# #             include_top=False ,
# #             input_shape=[*IMAGE_SIZE, 3]
# #         )
#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# Define early stopping condition
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

### Model2 - EfficientNet Model

In [ ]:
# # tf.keras.applications.efficientnet_v2.EfficientNetV2S(
# #     include_top=True,
# #     weights='imagenet',
# #     input_tensor=None,
# #     input_shape=None,
# #     pooling=None,
# #     classes=1000,
# #     classifier_activation='softmax',
# #     include_preprocessing=True
# # )

# with strategy.scope():
#     pretrained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
#         weights='imagenet',
#         include_top=False ,
#         input_shape=[*IMAGE_SIZE, 3]
#     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

From  the charts above shows that Efficient Net model performed poorly. It was far much worse than the InceptionV3 model. It won't be used to produce predictions. Lets try mobilenet models

### Model3 - MobileNetV2

In [ ]:
# # tf.keras.applications.mobilenet_v2.MobileNetV2(
# #     input_shape=None,
# #     alpha=1.0,
# #     include_top=True,
# #     weights='imagenet',
# #     input_tensor=None,
# #     pooling=None,
# #     classes=1000,
# #     classifier_activation='softmax',
# #     **kwargs
# # )

# with strategy.scope():
#     pretrained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
#         input_shape=[*IMAGE_SIZE, 3],
#         alpha=1.0,
#         include_top=False,
#         weights='imagenet',
#         input_tensor=None,
#         pooling=None,
#         classes=1000,
#         classifier_activation='softmax'
#     )
    
# #     pretrained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
# #         weights='imagenet',
# #         include_top=False ,
# #         input_shape=[*IMAGE_SIZE, 3]
# #     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

Mobile Net model has performed better than InceptionV3 and EfficientNet model. Let try to submit results before trying a few other models

## NASNet models
### Model4 - NASNetLarge

In [ ]:
# # tf.keras.applications.nasnet.NASNetLarge(
# #     input_shape=None,
# #     include_top=True,
# #     weights='imagenet',
# #     input_tensor=None,
# #     pooling=None,
# #     classes=1000,
# #     classifier_activation='softmax'
# # )

# with strategy.scope():
#     pretrained_model = tf.keras.applications.nasnet.NASNetLarge(
#         input_shape=[*IMAGE_SIZE, 3],
#         include_top=False,
#         weights='imagenet',
#         input_tensor=None,
#         pooling=None,
#         classes=1000,
#         classifier_activation='softmax'
#     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

It had an accuracy slightly lower than MobilenetV2

#### Model5 - NASNetMobile

In [ ]:
# with strategy.scope():
#     pretrained_model = tf.keras.applications.nasnet.NASNetMobile(
#         input_shape=[*IMAGE_SIZE, 3],
#         include_top=False,
#         weights='imagenet',
#         input_tensor=None,
#         pooling=None,
#         classes=1000,
#         classifier_activation='softmax'
#     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train 

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

## Regnet 

In [ ]:
# # tf.keras.applications.regnet.RegNetY320(
# #     model_name='regnety320',
# #     include_top=True,
# #     include_preprocessing=True,
# #     weights='imagenet',
# #     input_tensor=None,
# #     input_shape=None,
# #     pooling=None,
# #     classes=1000,
# #     classifier_activation='softmax'
# # )


# with strategy.scope():
# # tf.keras.applications.resnet.ResNet152(
# #     input_shape=[*IMAGE_SIZE, 3],
# #     include_top=False,
# #     weights='imagenet',
# #     input_tensor=None,
# #     pooling=None,
# #     classes=1000
# # )

#     pretrained_model = tf.keras.applications.regnet.RegNetY320(
#         model_name='regnety320',
#         input_shape=[*IMAGE_SIZE, 3],
#         include_top=False,
#         include_preprocessing=True,
#         weights='imagenet',
#         input_tensor=None,
#         pooling=None,
#         classes=1000,
#         classifier_activation='softmax'
#     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

## ResNet50

In [ ]:
# # tf.keras.applications.vgg16.VGG16(
# #     include_top=False,
# #     weights='imagenet',
# #     input_tensor=None,
# #     input_shape=[*IMAGE_SIZE, 3],
# #     pooling=None,
# #     classes=1000,
# #     classifier_activation='softmax'
# # )


# with strategy.scope():
#     pretrained_model = tf.keras.applications.vgg16.VGG16(
#         input_shape=[*IMAGE_SIZE, 3],
#         include_top=False,
#         weights='imagenet',
#         input_tensor=None,
#         pooling=None,
#         classes=1000,
#         classifier_activation='softmax'
#     )

#     pretrained_model.trainable = False
    
#     model = tf.keras.Sequential([
#         # base
#         pretrained_model,
        
#         # head
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.1),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
# )

# model.summary()

In [ ]:
# # Define training epochs
# EPOCHS = 20
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     callbacks=[es],
# )

In [ ]:
# ds_train

In [ ]:
# display_training_curves(
#     history.history['loss'],
#     history.history['val_loss'],
#     'loss',
#     211,
# )
# display_training_curves(
#     history.history['sparse_categorical_accuracy'],
#     history.history['val_sparse_categorical_accuracy'],
#     'accuracy',
#     212,
# )

## Best Models 
Many of the pretrained models have been used. Some have had great performance while others fair or poor performances. The top models are listed here;
* InceptionV3
* MobileNetV2
* NasNetLarge

In [ ]:
# from tensorflow.keras.applications import MobileNetV2, NASNetLarge
from tensorflow.keras.layers import Concatenate

with strategy.scope():
    # InceptionV3
    inception_model = tf.keras.applications.InceptionV3.InceptionV3(
        weights='imagenet', 
        include_top=False, 
        input_shape=[*IMAGE_SIZE, 3]
    )
    inception_model.trainable = False
    
    # MobileNetV2
    mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        weights='imagenet', 
        include_top=False, 
        input_shape=[*IMAGE_SIZE, 3]
    )
    mobilenet_model.trainable = False
    
    # NASNetLarge
    nasnet_model = tf.keras.applications.nasnet.NASNetLarge(
        weights='imagenet', 
        include_top=False, 
        input_shape=[*IMAGE_SIZE, 3]
    )
    nasnet_model.trainable = False
    
    # Concatenate models
    concatenated = Concatenate()([
        inception_model.output, 
        mobilenet_model.output, 
        nasnet_model.output
    ])
    
    global_avg_pooling = tf.keras.layers.GlobalAveragePooling2D()(concatenated)
    dense_layer = tf.keras.layers.Dense(len(CLASSES), activation='softmax')(global_avg_pooling)
    
    model = tf.keras.models.Model(inputs=[inception_model.input, mobilenet_model.input, nasnet_model.input], outputs=dense_layer)
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    

model.summary()

In [ ]:
stop 

In [ ]:
with strategy.scope():
    # 1. InceptionV3 Model
    inceptionv3_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False ,
        #input_shape=[*IMAGE_SIZE, 3]
    )

    inceptionv3_model.trainable = False
    
    # 2. MobileNetV2 Model
    mobileNetV2_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        # input_shape=[*IMAGE_SIZE, 3],
        alpha=1.0,
        include_top=False,
        weights='imagenet',
        classifier_activation='softmax'
    )
    
    mobileNetV2_model.trainable = False
    
    
    # 3. NasNetLarge Model
    nasNetLarge_model = tf.keras.applications.nasnet.NASNetLarge(
        #input_shape=[*IMAGE_SIZE, 3],
        include_top=False,
        weights='imagenet',
        classifier_activation='softmax'
    )
    
    nasNetLarge_model.trainable = False
    
    
    model = tf.keras.Sequential([
        # base
        #pretrained_model,
        # COMBINE THE MODELS
        model1_out = inceptionv3_model.output
        model1_out=GlobalAveragePooling2D()(model1_out)

        model2_out = mobileNetV2_model.output
        model2_out=GlobalAveragePooling2D()(model2_out)
        
        model3_out = nasNetLarge_model.output
        model3_out=GlobalAveragePooling2D()(model3_out)
        
        # head
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

model.summary()

In [ ]:
# Define training epochs
EPOCHS = 20
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[es],
)

In [ ]:
ds_train

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

# Submissions

In [ ]:
print('Computing predictions...')
test_ds = get_test_dataset(ordered=True)
test_images = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

In [ ]:
print('Generating submission.csv file...')
# Get image ids from test set and convert to unicode
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)